# LangChain Fundamentals & Agent Building
## Student Workbook

**Duration:** 45 minutes

**What you will learn:**
- What LangChain is and why it matters
- How to build your first LLM-powered agent
- Core concepts: Prompts, Tools, Agents
- Portfolio-ready code you can show employers

---

## How to Use This Notebook

- Follow along with the instructor's demonstrations
- Run the code cells when instructed
- Complete the exercises in the designated cells
- Check the troubleshooting section at the bottom if you get stuck

---

# Part 1: What is LangChain?

## The Problem LangChain Solves

**Without LangChain:**
- API calls are verbose
- Memory management is manual
- Error handling everywhere
- Tool integration requires custom code
- Result: 200+ lines of boilerplate for simple tasks

**With LangChain:**
- Simple LLM interface
- Built-in memory
- Error handling
- Tool framework
- Agent orchestration
- Result: 5-10 lines of clean code

## Architecture Overview

```
                    ┌─────────────┐
                    │   Your App  │
                    └──────┬──────┘
                           │
                ┌──────────┴──────────┐
                │    LangChain        │
                │  ┌────────────────┐ │
                │  │ Prompts        │ │
                │  │ Memory         │ │
                │  │ Chains         │ │
                │  │ Tools          │ │
                │  │ Agents         │ │
                │  └────────────────┘ │
                └──────────┬──────────┘
                           │
            ┌──────────────┼──────────────┐
        ┌───┴───┐   ┌──────┴─────┐  ┌────┴────┐
        │OpenAI │   │ Anthropic  │  │ Google  │
        │(GPT)  │   │ (Claude)   │  │(Gemini) │
        └───────┘   └────────────┘  └─────────┘
```

---

# Part 2: Setup

## Step 1: Install Required Libraries

In [ ]:
# Run this cell to install required packages
# You only need to do this once

!pip install langchain langchain-openai python-dotenv

## Step 2: Set Up Your API Key

**Option A: Using a .env file (recommended)**

1. Create a file called `.env` in your project folder
2. Add this line: `OPENAI_API_KEY=sk-your-key-here`
3. Run the cell below

**Option B: Direct entry (for this session only)**

1. Uncomment the second option in the cell below
2. Replace with your actual API key

**Need an API key?** Go to https://platform.openai.com/api-keys

In [ ]:
# SETUP: Import libraries and load API key
import os
from dotenv import load_dotenv

# Option A: Load from .env file
load_dotenv()

# Option B: Set directly (uncomment and add your key)
# os.environ["OPENAI_API_KEY"] = "sk-your-key-here"

# Check if key is loaded
if os.environ.get("OPENAI_API_KEY"):
    print("✓ API key loaded successfully")
else:
    print("✗ API key not found - see troubleshooting section")

---

# Part 3: Your First LLM Call

## Demo: Watch the Instructor

The instructor will show you how to:
- Import the LLM
- Create an instance
- Send a prompt
- Get a response

**Follow along by running the cell below:**

In [ ]:
# DEMO: Simple LLM Call

# Import the LLM
from langchain_openai import ChatOpenAI

# Create an instance
# temperature=0.7 means: how creative (0=factual, 1=creative)
llm = ChatOpenAI(
    model="gpt-4-turbo",
    temperature=0.7
)

# Send a prompt
prompt = "What is the best colour and why?"

# Get the response
response = llm.invoke(prompt)

# Print it
print("\n" + "="*60)
print("LLM RESPONSE:")
print("="*60)
print(response.content)
print("="*60)

## Key Concepts

| Concept | Meaning |
|---------|----------|
| **Model** | Which LLM to use (GPT-4, Claude, etc.) |
| **Temperature** | Creativity level (0 = precise, 1 = random) |
| **Prompt** | What you ask the LLM |
| **Response** | What the LLM sends back |

---

## Exercise 1: Write Your Own Prompt

**Instructions:**
1. Replace the placeholder text with your own question
2. Run the cell
3. See what the LLM responds

**Ideas for prompts:**
- "Why is Python better than JavaScript?"
- "What's a cool fact about AI?"
- "Tell me a funny joke about programming"
- "What should I build as my first AI project?"

In [ ]:
# EXERCISE 1: Write your own prompt

# TODO: Replace this with your own question
your_prompt = "YOUR QUESTION HERE"

# Run the LLM
response = llm.invoke(your_prompt)

# Display the result
print(f"Your prompt: {your_prompt}")
print(f"\nResponse: {response.content}")

---

# Part 4: Building Your First Agent

## What is an Agent?

An agent is an LLM that can use **tools** to solve problems:
- It **decides** which tool to use based on the question
- It reads the tool result
- It formulates an answer

**Key difference:**
- Without tools: LLM only knows what was in training data
- With tools: LLM can access real-time data, compute, and more

---

## Step 1: Define Tools

A tool is a Python function that the agent can call.

**Watch the instructor demonstrate, then run the cell:**

In [ ]:
from langchain.tools import tool

# Tool 1: Search Wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """
    Search Wikipedia for information about a topic.
    Use this to find facts and background information.
    """
    # Simulated response for demo purposes
    return f"Wikipedia results for '{query}': [Found detailed information about {query}]"

# Tool 2: Calculator
@tool
def calculate(expression: str) -> str:
    """
    Calculate a mathematical expression.
    Use this for maths problems, statistics, etc.
    """
    try:
        result = eval(expression)
        return f"Result: {expression} = {result}"
    except:
        return f"Error: Could not calculate {expression}"

print("✓ Tools defined:")
print("  - search_wikipedia")
print("  - calculate")

## Step 2: Create the Agent

Now we give these tools to the LLM and create an agent.

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain import hub

# Create the LLM
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# Get a pre-made prompt template for tool-using agents
prompt = hub.pull("hwchase17/openai-tools-agent")

# Define which tools the agent can use
tools = [search_wikipedia, calculate]

# Create the agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an executor (runs the agent)
executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True  # Shows what the agent is thinking
)

print("✓ Agent created and ready to use")

## Step 3: Run the Agent

Watch how the agent decides which tool to use!

In [ ]:
# Run the agent with a question
print("\n" + "="*70)
print("AGENT DEMO: Calculate something and explain it")
print("="*70)

result = executor.invoke({
    "input": "What is 15 * 12? Then search for what that number means."
})

print("\n" + "="*70)
print("FINAL ANSWER:")
print("="*70)
print(result["output"])
print("="*70)

## What Just Happened?

**Agent's thought process (from verbose output):**

1. THOUGHT: "User asked for 15*12 and then search for the result"
2. ACTION: Use 'calculate' tool with '15*12'
3. OBSERVATION: "Result: 15*12 = 180"
4. THOUGHT: "Now I should search for 180"
5. ACTION: Use 'search_wikipedia' with '180'
6. OBSERVATION: "Wikipedia results for '180': [Found...]"
7. FINAL ANSWER: "15*12 = 180. In history/math, 180 represents..."

**Key insight:** The agent DECIDED what to do. It wasn't programmed to search - it chose to!

---

# Part 5: HANDS-ON LAB
## Build a Colour-Arguing Agent
## Duration: 15 minutes

### Challenge

Build an agent that argues for a specific colour being the best.

The agent should:
1. Pick a colour (Red, Green, Blue, Yellow, Purple, etc.)
2. Have 2-3 tools that help it make arguments
3. Make a passionate argument for that colour

### Success Criteria

- Agent runs without errors
- Agent uses at least one tool
- Agent produces an argument for its colour
- Response is 3-5 sentences long

### Ideas for Tools

- Search for facts about the colour
- Find where the colour appears in nature
- Find cultural significance
- Search for psychology facts about the colour
- Find famous things that are that colour

---

### Starter Template

**Fill in the TODO sections:**

In [ ]:
from langchain.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain import hub

# ============================================
# STEP 1: Define Your Tools
# ============================================

# TODO: Pick your colour
MY_COLOUR = "TODO: Choose: Red, Green, Blue, Yellow, Purple, etc."

# TODO: Define Tool 1
# Make up a tool that finds facts about your colour
@tool
def search_colour_facts(topic: str) -> str:
    """
    Search for interesting facts about a colour.
    """
    # TODO: Return a more helpful response
    return f"Found facts: {MY_COLOUR} is associated with {topic}"

# TODO: Define Tool 2
# Make up another tool - think of something useful
@tool
def search_colour_usage(domain: str) -> str:
    """
    Search for where your colour is used.
    """
    # TODO: Return a more helpful response
    return f"In {domain}, {MY_COLOUR} is used for important things"

print(f"✓ Defined tools for {MY_COLOUR}")

In [ ]:
# ============================================
# STEP 2: Create the Agent
# ============================================

# TODO: Create LLM instance
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.7)

# TODO: Get the agent prompt template
prompt = hub.pull("hwchase17/openai-tools-agent")

# TODO: List your tools in a list
tools = [search_colour_facts, search_colour_usage]

# TODO: Create the agent
agent = create_tool_calling_agent(llm, tools, prompt)

# TODO: Create executor
executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

print(f"✓ Agent ready")

In [ ]:
# ============================================
# STEP 3: Create Your Prompt
# ============================================

# TODO: Write a prompt that makes the agent argue for your colour
# The more specific, the better the arguments!

# Examples:
# "You are a passionate RED enthusiast. Argue why RED is the best colour!"
# "You love GREEN. Why is GREEN superior to all other colours? Use facts!"
# "BLUE is clearly the best colour. Prove it using your tools."

my_prompt = f"""
TODO: Write your prompt here.
Remember to:
- Set a character or role
- Ask the agent to use tools
- Request 3-4 strong reasons
- Make it fun and persuasive
"""

print(f"✓ Prompt created")
print(f"\nPrompt preview:\n{my_prompt}")

In [ ]:
# ============================================
# STEP 4: Run Your Agent!
# ============================================

print("\n" + "="*70)
print(f"{MY_COLOUR.upper()} AGENT - MAKING ARGUMENTS")
print("="*70 + "\n")

result = executor.invoke({"input": my_prompt})

print("\n" + "="*70)
print("FINAL ARGUMENT:")
print("="*70)
print(result["output"])
print("="*70)

### Extensions (For Fast Finishers)

Try one of these to make your agent more advanced:

**Extension 1: Add a 3rd Tool**

```python
@tool
def search_psychology(aspect: str) -> str:
    """Find psychological facts about your colour"""
    return f"Psychology: {MY_COLOUR} makes people feel..."
```

**Extension 2: Make Your Agent Funny**

Add to your prompt: "Be sarcastic and funny while arguing"

**Extension 3: Have Agent Argue Against Another Colour**

Add to your prompt: "Also explain why RED is worse than your colour"

**Extension 4: Change the Temperature**

```python
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.9)  # More creative
```

**Extension 5: Add Personality**

Change prompt to: "You are a teenager arguing passionately..." or "You are a professional colour scientist arguing..."

---

# Part 6: Recap

## What You Learned

**LangChain basics:**
- What LangChain is and why it matters
- How to import and use it

**LLMs in Python:**
- How to create an LLM instance
- How to call it with a prompt
- How to get and use the response

**Tools:**
- What tools are (functions the agent can call)
- How to define tools with the @tool decorator
- How tools give agents real-world capabilities

**Agents:**
- What agents are (LLMs with decision-making)
- How agents decide which tools to use
- How to create and run agents
- How agents reason and explain their thinking

---

## Real-World Applications

What your code can do:
- Chat with documents (agent reads PDFs and answers questions)
- Autonomous research (agent finds info from multiple sources)
- Code debugging (agent analyses code and suggests fixes)
- Data analysis (agent processes data and creates reports)
- Customer service (agent handles customer questions with tools)
- Task automation (agent decides what actions to take)

---

## Key Takeaway

**You just built what AI engineers do daily.**

- Companies like OpenAI, Anthropic, and Google use this exact pattern
- This is day 2-3 work at a real AI startup
- You have portfolio-ready code
- Employers specifically look for this skill

---

# Troubleshooting & FAQ

## API Key Issues

**Problem: "API key not found"**

Solutions:
1. Make sure you created a `.env` file with `OPENAI_API_KEY=sk-...`
2. Check the file is in your project root directory
3. Restart your Jupyter kernel after creating the file

**Problem: "Invalid API key"**

Solutions:
1. Go to https://platform.openai.com/api-keys
2. Generate a new API key
3. Copy the entire key (including sk- prefix)
4. Paste it into your `.env` file

---

## LLM Call Issues

**Problem: "Rate limit exceeded"**

Solutions:
1. You have made too many API calls
2. Wait 1-2 minutes before trying again
3. Use a cheaper model: `model="gpt-3.5-turbo"`

**Problem: "Model not available"**

Solutions:
1. Try a different model:
   ```python
   llm = ChatOpenAI(model="gpt-3.5-turbo")
   ```
2. Check your OpenAI account has access to the model

**Problem: "Connection timeout"**

Solutions:
1. Check your internet connection
2. Try again in a few seconds
3. Check OpenAI status page: https://status.openai.com/

---

## Agent Issues

**Problem: Agent doesn't use tools**

Solutions:
1. Make your prompt more specific
2. Add "Use your tools to find facts" to your prompt
3. Check that tools are defined correctly

**Problem: Agent keeps repeating itself**

Solutions:
1. Reduce temperature: `temperature=0.3` (more deterministic)
2. Simplify your prompt
3. Check tool outputs are helpful

**Problem: Agent takes too long to run**

Solutions:
1. This is normal with multiple tool calls
2. Try using fewer tools
3. Use `gpt-3.5-turbo` instead of `gpt-4-turbo`

**Problem: ImportError when importing from langchain**

Solutions:
1. Make sure you ran the installation cell first
2. Restart your Jupyter kernel
3. Check you typed the import statement correctly

---

## General Questions

**Q: Why do I need temperature?**

A: Temperature controls creativity. Use 0 for precise tasks (like agents), use 0.7+ for creative tasks.

**Q: What does verbose=True do?**

A: It shows you the agent's thinking process, so you can see which tools it used and why.

**Q: How much does this cost?**

A: OpenAI charges per token. Each demo call costs pennies. You get free trial credits.

**Q: Can I use a different LLM?**

A: Yes! LangChain works with Claude, Gemini, and others. Change the import and model accordingly.

**Q: What's the difference between temperature 0.5 and 0.7?**

A: 0.5 is more factual, 0.7 is more creative. There's no perfect number, experiment to find what works.

---

# Self-Assessment

After this section, tick the boxes below if you can do each thing:

## Understanding

- [ ] I can explain what LangChain is
- [ ] I understand why companies use it
- [ ] I know what tools and agents are
- [ ] I can explain how agents make decisions

## Doing

- [ ] I can install LangChain
- [ ] I can create an LLM instance
- [ ] I can call the LLM with a prompt
- [ ] I can define custom tools
- [ ] I can create an agent
- [ ] I can run an agent and interpret output

## Building

- [ ] I can build an agent for my own use case
- [ ] I can add custom tools
- [ ] I can optimise prompts for better results

## Challenge Questions

Can you answer these?

1. "Can I explain what an agent is to someone else?"
2. "Can I create a tool and add it to an agent?"
3. "Can I write a prompt that makes an agent do what I want?"
4. "Can I debug an agent that is not working?"
5. "Can I think of a real problem I could solve with an agent?"

If you can answer YES to all 5, you have mastered this section! ✅

---

# Resources for Learning More

## Official Documentation

- **LangChain Docs:** https://python.langchain.com/
- **LangChain Agents:** https://python.langchain.com/docs/modules/agents/
- **Tool Calling:** https://python.langchain.com/docs/modules/agents/tools/

## Great Tutorials

- LangChain Getting Started (quick intro to basics)
- Build Your First Agent (step-by-step tutorial)
- Advanced Tool Building (creating complex tools)

## Practice Projects

1. Email Assistant (agent reads emails and summarises)
2. Code Helper (agent understands code and explains it)
3. Research Bot (agent gathers info from multiple sources)
4. Task Planner (agent creates and executes task plans)
5. FAQ Answerer (agent finds answers in knowledge base)

---

# Portfolio Tips

## Make Your Project Stand Out

**On GitHub:**
1. Create a clear README explaining what your agent does
2. Include setup instructions so others can run it
3. Show example outputs
4. Explain your design choices
5. Link to relevant job postings that need this skill

**In interviews:**
- Walk through your agent code
- Explain why you chose specific tools
- Discuss limitations and how to overcome them
- Show how you debugged problems
- Share what you would build next

**Things that impress employers:**
- Clean, documented code
- Real-world problem solving
- Multiple tools and agents
- Error handling
- Performance optimisation
- Testing

---

# Final Thoughts

## What You Accomplished Today

You started knowing nothing about LangChain.

You end knowing how to build AI agents.

You have portfolio-ready code.

You understand production-level concepts.

---

## The Real Opportunity

This skill is in huge demand right now.

**Job market:**
- 80% of AI engineering jobs require this
- Starting salary: GBP 50-70k
- With 2 years experience: GBP 90-130k
- Companies desperate to hire people who know this

**What makes you valuable:**
- Most people can use ChatGPT
- You can BUILD AI SYSTEMS
- You understand the architecture
- You know production patterns
- Employers want YOU

---

## One More Thing

**The best way to learn:** Build something real.

Do not just follow tutorials. Pick a problem YOU care about and solve it with an agent.

**Ideas:**
- Problem from your own life
- Something that bothers you about your work
- A tool you wish existed
- An automation that would save time

Build it. Deploy it. Show it off.

**That is how you become an AI engineer.**

---

### Questions?

If anything is unclear, now is the time to ask!

The instructor is here to help. This is your foundation.

Let us keep the energy high for what comes next! 🚀